In [1]:
import os
import numpy as np
%pip install SimpleITK
%pip install imutils
%pip install opencv-python
import SimpleITK as sitk
from imutils import paths

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
# function to resample the image
def resampling(image, spacing = (0.5, 0.5, 3), is_label = False):
    Dimension = image.GetDimension()
 # get original spacing and size
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()

    # convert our z, y, x convention to SimpleITK's convention
    out_spacing = list(spacing)[::-1]
        # calculate output size in voxels
    out_size = [
        int(np.round(
            size * (spacing_in / spacing_out)
        ))
        for size, spacing_in, spacing_out in zip(original_size, original_spacing, out_spacing)
    ]

    Resampler_func = sitk.ResampleImageFilter()
    Resampler_func.SetReferenceImage(image)
    Resampler_func.SetTransform(sitk.AffineTransform(Dimension))
    Resampler_func.SetOutputSpacing(out_spacing) 
    Resampler_func.SetSize(out_size)
    Resampler_func.SetOutputDirection(image.GetDirection())
    Resampler_func.SetOutputOrigin(image.GetOrigin())
    Resampler_func.SetDefaultPixelValue(image.GetPixelIDValue())
    if is_label:
        Resampler_func.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        Resampler_func.SetInterpolator(sitk.sitkBSpline)

    ResImage = Resampler_func.Execute(image)
    return ResImage

In [6]:
def crop_or_pad(image):
    size = (300, 300, 16)
    # set identity operations for cropping and padding
    rank = len(size)
    padding = [[0, 0] for _ in range(rank)]
    slicer = [slice(None) for _ in range(rank)]
    shape = image.GetSize()
    # for each dimension, determine process (cropping or padding)
    for i in range(rank):
        if shape[i] < size[i]:
            # set padding settings
            padding[i][0] = (size[i] - shape[i]) // 2
            padding[i][1] = size[i] - shape[i] - padding[i][0]
        else:
            # create slicer object to crop image
            idx_start = int(np.floor((shape[i] - size[i]) / 2.))
            idx_end = idx_start + size[i]
            slicer[i] = slice(idx_start, idx_end)

    # crop and/or pad image
    if isinstance(image, sitk.Image):
        pad_filter = sitk.ConstantPadImageFilter()
        pad_filter.SetPadLowerBound([pad[0] for pad in padding])
        pad_filter.SetPadUpperBound([pad[1] for pad in padding])
        return pad_filter.Execute(image[tuple(slicer)])
    else:
        return np.pad(image[tuple(slicer)], padding)


In [7]:
def slicing(image):
    size_z = image.GetSize()[2]
    sliced_images = []
    # Extract the slice from the MRI volume
    num_slices = 16

    # Compute the slice thickness
    slice_thickness = int(size_z / num_slices)

    # Loop through each slice in the z-axis and save it as a separate image
    for i in range(num_slices):
    # Compute the slice index
        slice_index = i * slice_thickness

        # Extract the slice from the input image
        slice_image = image[:, :, slice_index:slice_index+slice_thickness]

        # Resample the slice to 300x300 pixels
        image_slice_resampled = sitk.Resample(slice_image, [300, 300], sitk.Transform(), sitk.sitkLinear, slice_image.GetOrigin(), [slice_image.GetSpacing()[0], slice_image.GetSpacing()[1]], slice_image.GetDirection(), 0.0, slice_image.GetPixelID())
        sliced_images.append(image_slice_resampled)
    return sliced_images

In [11]:
def flipping_image(image):
    # use sitk Flip function, where the True, False, false specifies to flip horizontally
    flipped_image = sitk.Flip(image, [True, False, False])
    return flipped_image


In [9]:
def normalized(image):
    image_array = sitk.GetArrayFromImage(image)
    
    # Calculate the mean and standard deviation of the pixel values
    mean = np.mean(image_array)
    std = np.std(image_array)
    
    # Normalize the pixel values using z-score
    normalized_array = (image_array - mean) / std
    
    # Create a new SimpleITK image from the normalized pixel array
    normalized_image = sitk.GetImageFromArray(normalized_array)
    normalized_image.CopyInformation(image)
    
    return normalized_image

In [15]:
# path to all the volumes and masks
dataPaths = [r"E:\projects\image_processing\picai_public_images_fold0", 
            r"E:\projects\image_processing\picai_public_images_fold1",
             r"E:\projects\image_processing\picai_public_images_fold2",
            r"E:\projects\image_processing\picai_public_images_fold3",
            r"E:\projects\image_processing\picai_public_images_fold4",
            r"E:\projects\image_processing\picai_labels\anatomical_delineations\whole_gland\AI\Bosma22b",
            r"E:\projects\image_processing\picai_labels\csPCa_lesion_delineations\AI\Bosma22a"]

sliced_train_data = []
sliced_valid_data = []
sliced_test_data = []
sliced_mask_data = []
trained_data = []
test_data = []
validation_data = []
mask_data = []

# iterating over all the folders including both masks and volumes
for dataPath in dataPaths:
    # pathToImages include path to all the files and images within the dataPath folder
    pathToImages = list(paths.list_files(dataPath))
    # iterarting and working on each image
    for image in pathToImages[:20]:
        if image[-7: ] != "cor.mha" and image[-7: ] != "sag.mha":  # since we are not working on "cor.mha" and "sag.mha files", we added this condition
            # 1. resample the image
            # 2. crop the image
            # 3. flip the image and save both original and flipped image
            # 4. normalized both flipped and original image
            # 5. sliced both flipped and original image and added it in an array
            # 6. write the image and append it in training, testing and validation array accordingly.
            img = sitk.ReadImage(image)
            resampledImage = resampling(img)
            croppedImage = crop_or_pad(resampledImage)
            flipImage = flipping_image(croppedImage)
            fold =  dataPath[len(dataPath)-1]
            if fold == "1" or fold == "2" or fold == "4":
                croppedImage = normalized(croppedImage)
                flipImage = normalized(flipImage)
                sliced_train_data.append(slice(croppedImage))
                sliced_train_data.append(slice(flipImage))
                sliced_no = 0
                for image in sliced_train_data: 
                    path = image + "_" + str(sliced_no)
                    sliced_no += 1
                    sitk.WriteImage(image,  path)
                    trained_data.append(image)
            if fold == "3":
                croppedImage = normalized(croppedImage)
                flipImage = normalized(flipImage)
                sliced_valid_data.append(slice(croppedImage))
                sliced_valid_data.append(slice(flipImage))
                sliced_no = 0
                for image in sliced_valid_data:
                    path = image + "_" + str(sliced_no)
                    sliced_no += 1
                    sitk.WriteImage(image,  path)
                    validation_data.append(image)
            if fold == "0":
                croppedImage = normalized(croppedImage)
                flipImage = normalized(flipImage)
                sliced_test_data.append(slice(croppedImage))
                sliced_test_data.append(slice(flipImage))
                sliced_no = 0
                for image in sliced_test_data:
                    path = image + "_" + str(sliced_no)
                    sliced_no += 1
                    sitk.WriteImage(image,  path)
                    test_data.append(image)
            if fold == "b" or fold == "a":
                sliced_mask_data.append(slice(croppedImage))
                sliced_mask_data.append(slice(flipImage))
                sliced_no = 0
                for image in sliced_mask_data:
                    path = image + "_" + str(sliced_no)
                    sliced_no += 1
                    sitk.WriteImage(image,  path)
                    mask_data.append(image)
                
            
        
            

               
        


MemoryError: 

In [ ]:
import os
import numpy as np
import SimpleITK as sitk
from imutils import paths

# divide data into training, testing and validation.
train_data = []
test_data = []
validation_data = []

train_data = [r"picai_folder/picai/picai_public_images_fold1",
             r"picai_folder/picai/picai_public_images_fold2",
             r"picai_folder/picai/picai_public_images_fold4"]
validation_data = [r"picai_folder/picai/picai_public_images_fold3"]
test_data = [r"picai_folder/picai/picai_public_images_fold0"]
dataPaths_masks = [r"picai_folder/picai/picai_labels/anatomical_delineations/whole_gland/AI/Bosma22b",
                   r"picai_folder/picai/picai_labels/csPCa_lesion_delineations/AI/Bosma22a"]
train_ids = []

# getting ids/ name of all the files in these folders.
train_ids_1 = next(os.walk(train_data[0]))[1]
train_ids.append(train_ids_1)
train_ids_2 = next(os.walk(train_data[1]))[1]
train_ids.append(train_ids_2)
train_ids_3 = next(os.walk(train_data[2]))[1]
train_ids.append(train_ids_3)

sorted_train_ids = train_ids.sort(key = int)
    
test_ids = next(os.walk(test_data))[1]
validation_ids = next(os.walk(validation_data))[1]

prostate_masks_ids = next(os.walk(dataPaths_masks[0]))[1]
lesion_mask_ids = next(os.walk(dataPaths_masks[1]))[1]

# initialising three models for training.
X_train_model1 = np.zeroes((len(train_ids), 300, 300, 1), dtype= np.uint8)
Y_train_model1 = np.zeroes((len(train_ids), 300, 300, 1), dtype = np.bool)

X_train_model2 = np.zeroes((len(train_ids), 300, 300, 1), dtype= np.uint8)
Y_train_model2 = np.zeroes((len(train_ids), 300, 300, 1), dtype = np.bool)

X_train_model3 = np.zeroes((len(train_ids), 300, 300, 1), dtype= np.uint8)
Y_train_model3 = np.zeroes((len(train_ids), 300, 300, 1), dtype = np.bool)

i_1 = 0
i_2 = 0
i_4 = 0

# map each image with the corresponding mask.
for n, id_ in enumerate(prostate_masks_ids):
    mask_id = id_.split("_") # to get id of the patient from the path of masks.

    # to get id from path of volume images in folder1
    path_to_images_fold1 = list(paths.list_files(train_data[0]))[i_1]
    path_to_images_fold1_split = path_to_images_fold1.split("/")
    id_1 = path_to_images_fold1_split[-1]

    # to get id from path of volume images in folder2
    path_to_images_fold2 = list(paths.list_files(train_data[1]))[i_2]
    path_to_images_fold2_split = path_to_images_fold2.split("/")
    id_2 = path_to_images_fold2_split[-1]

    # to get id from path of volume images in folder4
    path_to_images_fold4 = list(paths.list_files(train_data[2]))[i_4]
    path_to_images_fold4_split = path_to_images_fold4.split("/")
    id_4 = path_to_images_fold4_split[-1]

    # if image is in folder 1
    if mask_id[0] == id_1:
        for image in path_to_images_fold1[id_1 * 5:id_1 + 5]: # run a loop to get images for that user
            # if image is of type "adc.mha" map it with prostate gland mask
            if image[-7: ] == "adc.mha": 
                img = sitk.ReadImage(image)
                X_train_model1[n] = img
                prostate_gland_image = dataPaths_masks[0] + "\\" + id_ 
                Y_train_model1[n] = sitk.ReadImage(prostate_gland_image)

            # if image is of type "hbv.mha" map it with cancer lesion mask
            if image[-7: ] == "hbv.mha":
                img = sitk.ReadImage(image)
                X_train_model2[n] = img
                cancer_lesion_image = dataPaths_masks[1] + "\\" + id_ 
                Y_train_model2[n] = sitk.ReadImage(cancer_lesion_image)

            # if image is of type "t2w.mha" map it with cancer lesion mask  
            if image[-7: ] == "t2w.mha":
                img = sitk.ReadImage(image)
                X_train_model3[n] = img
                cancer_lesion_image = dataPaths_masks[1] + id_ 
                Y_train_model3[n] = sitk.ReadImage(cancer_lesion_image)

        i_1 = i_1 + 1

    # if image is in folder 2
    elif mask_id[0] == id_2:
        for image in path_to_images_fold1[id_1:id_1 + 5]:
            if image[-7: ] == "adc.mha":
                img = sitk.ReadImage(image)
                X_train_model1[n] = img
                prostate_gland_image = dataPaths_masks[0] + "\\" + id_ 
                Y_train_model1[n] = sitk.ReadImage(prostate_gland_image)
            if image[-7: ] == "hbv.mha":
                img = sitk.ReadImage(image)
                X_train_model2[n] = img
                cancer_lesion_image = dataPaths_masks[1] + "\\" + id_ 
                Y_train_model2[n] = sitk.ReadImage(cancer_lesion_image)
            if image[-7: ] == "t2w.mha":
                img = sitk.ReadImage(image)
                X_train_model3[n] = img
                cancer_lesion_image = dataPaths_masks[1] + "\\" + id_ 
                Y_train_model3[n] = sitk.ReadImage(cancer_lesion_image)

        i_2 = i_2 + 1

    # if image is in folder 4
    elif mask_id[0] == id_4:
        for image in path_to_images_fold1[id_1:id_1 + 5]:
            if image[-7: ] == "adc.mha":
                img = sitk.ReadImage(image)
                X_train_model1[n] = img
                prostate_gland_image = dataPaths_masks[0] + "\\" + id_ 
                Y_train_model1[n] = sitk.ReadImage(prostate_gland_image)
            if image[-7: ] == "hbv.mha":
                img = sitk.ReadImage(image)
                X_train_model2[n] = img
                cancer_lesion_image = dataPaths_masks[1] + "\\" + id_ 
                Y_train_model2[n] = sitk.ReadImage(cancer_lesion_image)
            if image[-7: ] == "t2w.mha":
                img = sitk.ReadImage(image)
                X_train_model3[n] = img
                cancer_lesion_image = dataPaths_masks[1]  + "\\" + id_ 
                Y_train_model3[n] = sitk.ReadImage(cancer_lesion_image)
        i_4 = i_4 + 1
        


In [ ]:
import tensorflow as tf
import tenserflow.keras.layers as K

def dice_coefficient(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1. - dice_coefficient(y_true, y_pred)


def unet(input_shape=(300, 300, 1), num_classes=1):
    inputs = tf.keras.layers.Input(shape=input_shape)
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
    # Downsampling
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(s)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = tf.keras.layers.Dropout(0.5)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)

    # Upsampling
    up6 = tf.keras.layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    concat6 = tf.keras.layers.concatenate([drop4, up6])
    conv6 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat6)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = tf.keras.layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    concat7 = tf.keras.layers.concatenate([conv3, up7])
    conv7 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat7)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = tf.keras.layers.Conv2DTranspose(128, 2, strides=(2. 2), padding= 'same', kernel_initializer='he_normal')(conv7)
    concat8 = tf.keras.layers.concatenate([conv2, up8])
    conv8 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat8)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = tf.keras.layers.Conv2DTranspose(64, 2, strides=(2. 2), padding= 'same', kernel_initializer='he_normal')(conv8)
    concat9 = tf.keras.layers.concatenate([conv1, up9])
    conv9 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat9)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)

    # Output
    outputs =  tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    # Model
    model = tf.keras.Model([inputs], [outputs])
    return model


model = unet()
model.compile(optimizer= 'adam', loss = dice_loss, metrics = [dice_coefficient])
model.summary()
checkpointers = tf.keras.callbacks.ModelCheckpoint('model_for_prostate.png', verbose = 1, save_best_only= True)
    
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]
    
results = model.fit(X_train_model1, Y_train_model1, validation_split=0.1, batch_size= 16, epochs=25, callbacks=callbacks)

